In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/DynaTrainCDD/DynaTrainCDD')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import pandas as pd
from scipy.stats import percentileofscore
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods

### dynamiccdd

In [ ]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/DynaTrainCDD/DynaTrainCDD')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import pandas as pd
from scipy.stats import percentileofscore
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods
from collections import deque
from itertools import islice
from collections import OrderedDict
from PrefixTreeCDD.PrefixTreeClass import PrefixTree
import PrefixTreeCDD.settings as settings
from PrefixTreeCDD.CDD import Window
from skmultiflow.drift_detection import ADWIN, PageHinkley
import math
from numpy import log as ln
import os
from pathlib import Path
from datetime import datetime

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'


def recCurve(a, b, x):
    return a * math.log(x+1, 10)**(b * math.log(x+12, 10)) + 5000


def recCurve2(x):
    return -2e-5*x**2 + 1000


def recCurveFinal(x,a,u_max):
    if x == 0:
        return 1
    return 1 - math.exp(a*(x/u_max-1))


def store_results(file, results):
    with open(file, "w") as fout:
        for r in results:
            fout.write(",".join([str(r_i) for r_i in r]) + "\n")


def store_resultsa(file, results):
    with open(file, "a") as fout:
        for r in results:
            fout.write(",".join([str(r_i) for r_i in r]) + "\n")


def store_timings(file, timings):
    with open(file, "w") as fout:
        for t in timings:
            fout.write(str(t) + "\n")


def store_timing(file, timing):
    with open(file, "w") as fout:
        fout.write(str(timing) + "\n")


def store_timinga(file, timing):
    with open(file, "a") as fout:
        fout.write(str(timing) + "\n")


def run(config, file, dataName, updateInt, updateWindow, trainPerc=0):

    # df = pd.DataFrame(batch_accuracy)
    datetime_str= datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    # Define the folder to save the CSV files
    output_folder_results =Path(__file__).parent/"results"/f"{Path(config.get_dataset()).stem}_rbs{config.get_recent_buffer_size()}_hbs{config.get_hard_buffer_size()}_{datetime_str}"
    if output_folder_results.exists():
        print(f"Folder {output_folder_results} already exists. Results will be overwritten.")
    output_folder_results.mkdir(parents=True, exist_ok=True)
    print(output_folder_results)

    #a = 0.1
    delay_val = 490 #MUST BE LESS THAN U_MAX! has been 1250
    u_min = 20 #has been 200
    u_max = 75 #has been 1500
    t_min = 20 #has been 500
    t_max = 75 #has been 750
    # u_min = 20 #has been 200
    # u_max = 50 #has been 1500
    # t_min = 50 #has been 500
    # t_max = 75 #has been 750
    a = ln(0.2) / ((delay_val /u_max) - 1)
    updateArray = range(u_min, u_max)
    trainArray = range(t_max, t_min, -1)
    print('updateArray', updateArray, 'trainArray', trainArray)
    file = file
    data = pd.read_csv(file, low_memory=False)
    timeformat = "%Y-%m-%d %H:%M:%S"
    numEvents = data.shape[0]
    print("Num events is {}".format(numEvents))

    d = Data(dataName,
             LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                     activity_attr='event', convert=False))
    d.logfile.keep_attributes(['event', 'role', 'completeTime'])
    m = Methods.get_prediction_method("SDL")
    s = setting.STANDARD
    trainPerc = trainPerc
    s.train_percentage = trainPerc * 100
    # # #
    d.prepare(s)
    # d.create_batch("normal", timeformat)
    is_written = 0
    print("Test Context Data")
    print(d.test_orig.contextdata)
    basic_model = m.train(d.train,{"epochs": 0, "early_stop": 10})

    #print("Runtime %s:" % m, time.time() - start_time)
    res = m.test(basic_model, d.test_orig)
    
    store_results(str(output_folder_results)+"/%s_%s_normal.csv" % (m.name, d.name), res)
    
    connect_symbol="-"
    if '/' in d.logfile.get_data()['completeTime'][0]:
        connect_symbol='/'
    formats = [f"%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S%z", f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S',f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S.%f']

    for timeformat in formats:
        try:
            pd.to_datetime(d.logfile.get_data()['completeTime'], format=timeformat,exact=True)
            print(d.logfile.get_data()['completeTime'])
            print("The time format is:", timeformat)
            break
        except ValueError:
            continue
    print('timeformat',timeformat)
    d.add_data_to_test_orig([17.5,50],timeformat,config.get_ratio())

    print("Baseline Complete")
    updateInt = updateInt
    updateWindow = updateWindow
    window_size = 10
    tree_size = 1000
    decay_lambda = 0.25
    noise = 1
    settings.init()
    endEventsDic = dict()
    window = Window(initWinSize=window_size)

    lastEvents = data.groupby(['case']).last()
    for _, row in lastEvents.iterrows():
        endEventsDic[_] = [str(row['event']), row['completeTime']]

    caseList = []  # Complete list of cases seen
    Dcase = OrderedDict()  # Dictionary of cases that we're tracking.
    print("You are here")
    tree = PrefixTree(pruningSteps=tree_size, noiseFilter=noise,
                      lambdaDecay=decay_lambda)  # Create the prefix tree with the first main node empty
    adwin = ADWIN()
    ph = PageHinkley()

    pruningCounter = 0  # Counter to check if pruning needs to be done
    traceCounter = 0  # Counter to create the Heuristics Miner model

    eventCounter = 0  # Counter for number of events
    currentNode = tree.root  # Start from the root node
    testInd = round(numEvents * trainPerc)
    prevDriftCounter = 0
    drifts = {}
    new_drifts = {}
    severity = -88
    start_time = time.time()

    for _, event in data.iterrows():
        caseList, Dcase, currentNode, pruningCounter, traceCounter, window = tree.insertByEvent(caseList, Dcase,
                                                                                                currentNode, event,
                                                                                                pruningCounter,
                                                                                                traceCounter,
                                                                                                endEventsDic, window)
        eventCounter += 1

        if window.cddFlag:  # If a complete new tree has been created
            if len(window.prefixTreeList) == window.WinSize:
                # Maximum size of window reached, start concept drift detection within the window
                temp_drifts = window.conceptDriftDetection(adwin, ph, eventCounter)
                window.WinSize = min(window.WinSize + 1, window.maxWindowSize)
                for i in temp_drifts.keys():
                    if i not in drifts.keys():
                        new_drifts[i] = temp_drifts[i]

                if len(window.prefixTreeList) == window.WinSize:  # If there was no drift detected within the window
                    window.prefixTreeList = deque(islice(window.prefixTreeList, 1, None))  # Drop the oldest tree

        #TODO: when drift occurs or other trigger is activated
        if len(list(drifts.keys()) + list(new_drifts.keys())) > prevDriftCounter and (_ > round(numEvents * trainPerc)):
            if len(list(drifts.keys())) >= 5:
                print('Take quantile of any new drifts that occur')
                #TODO: Take quantile of any new drifts that occur
                #list of drift severities
                drift_sevs = [drifts[i]['treeDist'] for i in list(drifts.keys())]
                new_drift_sevs = [new_drifts[i]['treeDist'] for i in list(new_drifts.keys())]
                severity = percentileofscore(a = drift_sevs, score = sum(new_drift_sevs)/len(new_drift_sevs))/100 #* 5000
                print("Severity is {}".format(severity))
                #calculate new severity using percentilesum(

            else:
                #TODO: max severity level
                severity = 1
                print("Max Severity level")
        elif (_ - testInd > updateInt) and (_ > round(numEvents * trainPerc)):
            #TODO: last drift greater than predetermined retrain frequency -> assign moderate severity
            print('last drift greater than predetermined retrain frequency -> assign moderate severity')
            if severity == -1 or severity == -88:
                print("Max update interval level")
                severity = 0.5
        elif severity == -88 and (_ > round(numEvents * trainPerc)) and len(list(drifts.keys()))>0:# last drift was < 5000 events ago....then update with the most recent available information
            print('last drift was < 5000 events ago....then update with the most recent available information')
            if _ - sorted([i for i in drifts.keys()])[-1] < 5000:
                severity = 1


        #TODO: become aware of drift severity above then make appropriate update below^^^
        #TODO: drift updating starts here, use this to set the updateInt from previous iterations
        if severity != -1 and severity != -88 and ((_ - testInd > updateInt) or len(list(new_drifts.keys())) > 0):
            severity = min(1, severity)
            print('severity',severity)

            #TODO: Update these values and create a list using rec_curve_final
            #severity outputs a score between 0 and 1
            #need to use the update and train array to actually convert to a meaningful update/train value
            print("Update val index value is {}".format(round(recCurveFinal(severity, a, u_max) * len(updateArray)) - 1))
            updateVal = min(updateArray[round(recCurveFinal(severity, a, u_max) * len(updateArray)) - 1], u_max)
            winVal = max(t_min, trainArray[round(recCurveFinal(severity, a, u_max) * len(trainArray)) - 1])
            updateVal, winVal = 49, 50
            updateInt, updateWindow = updateVal, winVal
            print("Update Value is {}".format(updateVal))
            print("Window Value is {}".format(winVal))
            #TODO: determine new values for updateInt and updateWindow before running below
            print("Performing Maintenance Update")
            print("Size of hyphen is {}".format(_))
            print("Highest index calculation on test is {}".format(_ - round(numEvents * trainPerc)))
            if updateWindow > _ - round(numEvents * trainPerc):
                print('updateWindow > _ - round(numEvents * trainPerc)')
                updateWindow = 0
            result, timing, basic_model = m.test_and_update_indices(basic_model, d, (_ - round(numEvents * trainPerc),
                                                                                     max(_ - round(
                                                                                         numEvents * trainPerc) - updateWindow, #maximally the total number in the test set to retrain
                                                                                         0)),
                                                                    testInd - round(numEvents * trainPerc),
                                                                    _ - round(numEvents * trainPerc),
                                                                    reset=False)
            testInd = _
            print("Model updated at event {}".format(str(_)))
            if is_written:
                store_resultsa(str(output_folder_results)+"/%s_%s_OTF_drift_%s.csv" % (m.name, d.name, 'dynamic'), result)
                store_timinga(str(output_folder_results/("%s_%s_OTF_drift_%s_time.csv" % (m.name, d.name, 'dynamic'))),
                              timing)
            else:
                store_results(str(output_folder_results)+"/%s_%s_OTF_drift_%s.csv" % (m.name, d.name, 'dynamic'), result)
                store_timing(str(output_folder_results/("%s_%s_OTF_drift_%s_time.csv" % (m.name, d.name, 'dynamic'))),
                             timing)
            is_written = 1
            #severity += 0.1

        #reset drifts for next run's analysis
        drifts = {**drifts, **new_drifts}
        prevDriftCounter += len(list(new_drifts.keys()))
        new_drifts = {}
        if severity >= 1:
            severity = -1




    # print("Size of hyphen is {}".format(_))
    # print("Highest index calculation on test is {}".format(_ - round(numEvents * trainPerc)))
    if _!=testInd:
        print("Performing Final Test")
        result, timing, basic_model = m.test_and_update_indices(basic_model, d, (_ - round(numEvents * trainPerc),
                                                                                max(_ - round(
                                                                                    numEvents * trainPerc) - updateWindow,
                                                                                    0)),
                                                                testInd - round(numEvents * trainPerc),
                                                                _ - round(numEvents * trainPerc),
                                                                reset=False)
        testInd = _

        print("Model updated at event {}".format(str(_)))
        if is_written:
            store_resultsa(str(output_folder_results/("%s_%s_OTF_drift_%s.csv" % (m.name, d.name, 'dynamic'))), result)
            store_timinga(str(output_folder_results/("%s_%s_OTF_drift_%s_time.csv" % (m.name, d.name, 'dynamic'))), timing)
        else:
            store_results(str(output_folder_results/("%s_%s_OTF_drift_%s.csv" % (m.name, d.name, 'dynamic'))), result)
            store_timing(str(output_folder_results/("%s_%s_OTF_drift_%s_time.csv" % (m.name, d.name, 'dynamic'))), timing)

    prediction_results_path = str(output_folder_results)+"/%s_%s_OTF_drift_%s.csv" % (m.name, d.name, 'dynamic')
    timing_path = str(output_folder_results/("%s_%s_OTF_drift_%s_time.csv" % (m.name, d.name, 'dynamic')))
    end_time = time.time()
    running_time = end_time - start_time
    print("Running time is {}".format(running_time))
    return output_folder_results, prediction_results_path, timing_path,running_time

# if __name__ == "__main__":
#     intervals = [1500] #has been 1500
#     windows = [750] #has been 750
#     files = ['/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/BPIC15_1_sorted_new.csv']#['Data/BPIC17_FULL.csv']#['Data/BPIC15_1_sorted_new.csv', 'Data/BPIC15_2_sorted_new.csv', 'Data/BPIC15_3_sorted_new.csv', 'Data/BPIC15_4_sorted_new.csv', 'Data/BPIC15_5_sorted_new.csv',
#              # 'Data/Helpdesk.csv', 'Data/BPIC11.csv', 'Data/BPIC12.csv']

#     #names = ['BPIC15_1_OTF_OPT']
#     names = ['BPIC17FULL'] #, 'BPIC15_2_OTF_OPT', 'BPIC15_3_OTF_OPT', 'BPIC15_4_OTF_OPT', 'BPIC15_5_OTF_OPT',
#               # 'Helpdesk_OTF_OPT', 'BPIC11_OTF_OPT', 'BPIC12_OTF_OPT']
#     for i in intervals:
#         for w in windows:
#             for f in range(len(files)):
#                 if i < 1000 and w < 1000:
#                     run(files[f], names[f], i, w)
#                 elif i >= 1000:
#                     run(files[f], names[f], i, w)


### define model, data

In [3]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/test_dataset/Helpdesk_mini.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

d = Data('Helpdesk_mini',
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'role', 'completeTime'])
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)
# d.create_batch("normal", timeformat)
is_written = 0
print("Test Context Data")
print(d.test_orig.contextdata)
basic_model = m.train(d.train,{"epochs": 0, "early_stop": 10})

Num events is 300
PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting role
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 0
Test: 300
Test Context Data
     level_0  index  event_Prev9  role_Prev9  completeTime_Prev9  case_Prev9  \
0          0    191            0           0                   0          59   
1          1    143            0           0                   0          43   
2          2    234            0           0                   0          70   
3          3     58            0           0                   0          17   
4          4    220            0           0                   0          67   
..       ...    ...          ...         ...                 ...         ...   
295      295    225            0           0                   0          68   
296      296    226            0           0                   0          68   
297      297    167            0           0                   0   

In [15]:
batch_inputs_lst,batch_expected_lst = m.test_and_update_indices(basic_model,d,(150,90),50,150,reset=False)

INFO:tensorflow:Assets written to: tmp_model/assets
Accuracy: 62.00%
train data: evWindow[1]:  90 evWindow[0]:  150 test data: prevTestInd:  50 currentTestInd:  150
num_batches: 3
Batch: 0
Batch: 1
Batch: 2


In [23]:
results, timings, model=m.test_and_update_indices(basic_model,d,(150,90),50,150,reset=False)

INFO:tensorflow:Assets written to: tmp_model/assets
Accuracy: 62.00%
train data: evWindow[1]:  90 evWindow[0]:  150 test data: prevTestInd:  50 currentTestInd:  150


In [22]:
results

In [17]:
for item in batch_inputs_lst: print(np.shape(item))

(20, 16)
(20, 16)
(20, 28)


In [15]:
print(type(batch_inputs),len(batch_inputs),np.shape(batch_inputs[0]),batch_inputs[0])

<class 'list'> 20 (16,) [0 1 6 0 1 6 7 0 0 1 6 1 0 1 6 0]


In [8]:
print(batch_expected)

[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]


In [13]:
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
for gs in range(2):
    with tf.GradientTape() as tape:
        y_pred = basic_model([tf.convert_to_tensor(np.asarray(row).reshape(-1,1)) for row in batch_inputs],training=True)
        y_sup = batch_expected
        recent_loss = loss_fn(y_sup, y_pred)
        total_loss = tf.reduce_sum(recent_loss)
        gradients = tape.gradient(total_loss, basic_model.trainable_variables)
        print('y_pred',y_pred,'y_sup',y_sup,recent_loss,'total_loss',total_loss)
        basic_model.optimizer.apply_gradients(zip(gradients, basic_model.trainable_variables))

y_pred tf.Tensor(
[[0.11963906 0.13621168 0.11888967 0.12126113 0.12076897 0.12262568
  0.13337098 0.12723288]
 [0.12007792 0.13274083 0.11783526 0.1209626  0.12197915 0.12245729
  0.13532451 0.12862247]
 [0.12012846 0.13331342 0.11801156 0.12121197 0.11964029 0.12327831
  0.1338708  0.1305452 ]
 [0.11930355 0.136541   0.11919087 0.11994054 0.12247207 0.12241466
  0.13331364 0.12682365]
 [0.12145756 0.13264136 0.11858703 0.12044381 0.11977818 0.12176019
  0.13643503 0.12889685]
 [0.11976922 0.13337825 0.11768399 0.12158215 0.11966705 0.12335523
  0.13357565 0.13098851]
 [0.11933742 0.1319848  0.11801521 0.1216635  0.12029705 0.12323031
  0.13352202 0.13194966]
 [0.1193942  0.13705905 0.11819307 0.12220512 0.12096544 0.12310756
  0.13243523 0.12664041]
 [0.119739   0.13690358 0.11862326 0.12057627 0.12019862 0.12287108
  0.13329177 0.12779647]
 [0.11981081 0.1325371  0.11929288 0.12035406 0.12071384 0.12282973
  0.13630348 0.1281581 ]
 [0.11984017 0.13323446 0.11752784 0.12179445 0.1207

In [16]:
model = basic_model
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)

In [23]:
print(type(model))
print(type(batch_inputs))
print(batch_inputs)
print('batch_expected',type(batch_expected),batch_expected)


<class 'tensorflow.python.keras.engine.training.Model'>
<class 'list'>
[array([0, 1, 6, 0, 1, 6, 7, 0, 0, 1, 6, 1, 0, 1, 6, 0]), array([0, 0, 1, 0, 0, 1, 6, 0, 0, 0, 1, 0, 0, 0, 1, 0]), array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 5, 5, 0, 5, 5, 5, 0, 0, 6, 6, 6, 0, 6, 6, 0]), array([0, 0, 5, 0, 0, 5, 5, 0, 0, 0, 6, 0, 0, 0, 6, 0]), array([0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 

In [25]:
for gs in range(10):
    with tf.GradientTape() as tape:
        y_pred = model([tf.convert_to_tensor(np.asarray(row).reshape(-1,1)) for row in batch_inputs],training=True)
        #print(type(y_pred),y_pred)
        y_sup = batch_expected
        #print(type(y_sup),y_sup)
        recent_loss = loss_fn(y_sup, y_pred)
        total_loss = tf.reduce_sum(recent_loss)
        gradients = tape.gradient(total_loss, model.trainable_variables)
        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [28]:
batch_input = [tf.convert_to_tensor(np.asarray(row).reshape(-1,1)) for row in batch_inputs][0]

In [26]:
m.test_and_update_indices(basic_model,d,(150,90),50,150,reset=False)

INFO:tensorflow:Assets written to: tmp_model/assets
Accuracy: 61.00%
train data: evWindow[1]:  90 evWindow[0]:  150 test data: prevTestInd:  50 currentTestInd:  150
num_batches: 3
Batch: 0


ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (1 total):
    * Tensor("inputs:0", shape=(16, 1), dtype=int64)
  Keyword arguments: {}

Expected these arguments to match one of the following 1 option(s):

Option 1:
  Positional arguments (1 total):
    * TensorSpec(shape=(None, 1), dtype=tf.float32, name='inputs')
  Keyword arguments: {}